# TD/ETD Comparison on "Random Walk"

This notebook contains some comparisons between TD and Emphatic TD on a simple problem under function approximation. 

We identify the solutions each algorithm converges to using the matrix operator equations for each algorithm, and compare them to the optimal approximation (as found by the least squares solution).

In [10]:
import numpy as np
import pandas as pd
from functools import reduce
from numpy import dot
from numpy.linalg import pinv

from features import *
from mdptools import *
from solvers import td_solution, etd_solution, exact_solution

In [11]:
from IPython.display import display

In [55]:
def report(P, R, phi_func, gm_func, lm_func, i_func):
    """Function for collating and comparing the various solutions."""
    states = state_vectors(pmat)
    nn = len(find_nonterminals(P))
    X = feature_matrix(states, phi_func)
    V = exact_solution(P, R, gm_func)
    
    # Best approximation (least squares)
    w_approx, *_ = np.linalg.lstsq(X, V)
    V_approx = np.dot(X, w_approx)
    E_approx = np.sum((V - V_approx)**2)/nn
    
    # TD
    w_td = td_solution(P, R, phi_func, gm_func, lm_func)
    V_td = np.dot(X, w_td)
    E_td = np.sum((V - V_td)**2)/nn
    
    # Emphatic TD fixed point
    w_etd = etd_solution(P, R, phi_func, gm_func, lm_func, i_func)
    V_etd = np.dot(X, w_etd)
    E_etd = np.sum((V - V_etd)**2)/nn
    
    dct = {"TD": {"weights": w_td, "MSE": E_td},
           "ETD": {"weights": w_etd, "MSE": E_etd},
           "Least-Squares": {"weights": w_approx, "MSE": E_approx},
           }
    
    dct = {"weights": 
            {
              "Least-Squares": w_approx, "TD": w_td, "ETD": w_etd,
            }, 
           "MSE": 
            {
              "Least-Squares": E_approx, "TD": E_td, "ETD": E_etd,
            },
           "values":
            {
                "Least-Squares": V_approx, "TD": V_td, "ETD": V_etd,
            },
           }
           
             
    
    df = pd.DataFrame(dct, 
                      index=["Least-Squares", "TD", "ETD"],
                      columns=["weights", "MSE", "values"],)
    
    # Additional Information
    print("Expected Reward:")
    print(R)
    
    print("Feature Matrix:")
    print(X)
    
    print("True Values:")
    print(V)
    
    print("Emphasis As Good or Better?:", (E_etd <= E_td))
    return df

# Random Walk

An environment which represents a random walk along a 1-D chain.

* Here, states are indexed from 0 to N-1, with states N and N-1 being the terminal states.
    * We refer to state `N-1` as the "rightmost" state, and state `N-2` as the "leftmost" state.
* In terminal states, the feature vector $x(s)$ is the zero vector, the reward $r(s)$ is zero, and $\gamma(s) = \lambda(s) = i(s) = 0$.
* The environment is undiscounted, so $\gamma(s) = 1$ for $s$ non-terminal.
* For these experiments, $\lambda(s) = 0$ for all states $s$.
* Interest is constant and uniform for each state; $i(s) = 1$ for $s$ non-terminal.

In [49]:
def random_walk_matrix(n, p=0.5):
    """
    The transition matrix for a random walk with `n` states (including 
    two terminal states).
    """
    ret = np.zeros((n,n))
    # terminal state transitions
    ret[-2:, -2:] = np.eye(2) 
    # transient states that can terminate
    ret[0,-2] = p       # left side of chain
    ret[0,1] = (1-p)
    ret[-3,-4] = p      # right side of chain
    ret[-3,-1] = (1-p)
    # handle rest of transient states
    for i in range(1, n-3):
        ret[i][i-1] = p 
        ret[i][i+1] = (1-p)
    return ret

In [71]:
# Common parts of problem specification
num_states = 7
pleft = 0.5
pmat = random_walk_matrix(num_states, p=pleft)
states = state_vectors(pmat)
indices = state_indices(pmat)
terminals = [as_tuple(s) for s in find_terminals(pmat)]
gmfunc = Constant(1.0, terminals=terminals)
lmfunc = Constant(0.0, terminals=terminals)
ifunc = Constant(1.0, terminals=terminals)

In [72]:
display(pmat)

array([[ 0. ,  0.5,  0. ,  0. ,  0. ,  0.5,  0. ],
       [ 0.5,  0. ,  0.5,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.5,  0. ,  0.5,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0.5,  0. ,  0.5,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0.5,  0. ,  0. ,  0.5],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  1. ]])

## Constant Feature, Unit Reward on Right-Termination

* reward for terminating in the rightmost state, `N-1`
    * r(s) = 1 for $(s, a, s') = (N-3, right, N-1)$
    * r(s) = 0 for all other transitions.
* x(s) = 1 for x non-terminal

In [73]:
rvec = np.zeros(num_states)
rvec[-3] = (1-pleft)
phi = Wrap(Bias(), terminals=terminals)

full_df = report(pmat, rvec, phi, gmfunc, lmfunc, ifunc)
df = full_df[["weights", "MSE"]]

display(full_df)

Expected Reward:
[ 0.   0.   0.   0.   0.5  0.   0. ]
Feature Matrix:
[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
True Values:
[ 0.16666667  0.33333333  0.5         0.66666667  0.83333333  0.          0.        ]
Emphasis As Good or Better?: True


,weights,MSE,values
Least-Squares,[0.5],0.055556,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0]"
TD,[0.5],0.055556,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0]"
ETD,[0.5],0.055556,"[0.5, 0.5, 0.5, 0.5, 0.5, 0.0, 0.0]"


In [70]:
rvec

array([array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.]),
       array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.]),
       array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.]),
       array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.]),
       array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0.0, 0.0], dtype=object)

## Constant Reward, Constant Feature

* r(s) = 1 for s non-terminal
* x(s) = 1 for x non-terminal

This effectively calculates the expected number of steps to termination.

In [57]:
rfunc =  Constant(1.0, terminals=terminals)
rvec = np.array([rfunc(s) for s in states])
phi = Wrap(Bias(), terminals=terminals)

full_df = report(pmat, rvec, phi, gmfunc, lmfunc, ifunc)
df = full_df[["weights", "MSE"]]

display(full_df)

Expected Reward:
[ 1.  1.  1.  1.  1.  0.  0.]
Feature Matrix:
[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
True Values:
[ 5.  8.  9.  8.  5.  0.  0.]
Emphasis As Good or Better?: True


,weights,MSE,values
Least-Squares,[7.0],2.8,"[7.0, 7.0, 7.0, 7.0, 7.0, 0.0, 0.0]"
TD,[5.0],6.8,"[5.0, 5.0, 5.0, 5.0, 5.0, 0.0, 0.0]"
ETD,[7.0],2.8,"[7.0, 7.0, 7.0, 7.0, 7.0, 0.0, 0.0]"


In [58]:
print(df.to_latex())

\begin{tabular}{llr}
\toprule
{} & weights &  MSE \\
\midrule
Least-Squares &   [7.0] &  2.8 \\
TD            &   [5.0] &  6.8 \\
ETD           &   [7.0] &  2.8 \\
\bottomrule
\end{tabular}



## Increasing Reward, Constant Feature

* r(s) = s+1 for s non-terminal
* x(s) = 1 for x non-terminal

In [61]:
_rfunc = lambda x: basis2int(x) + 1
rfunc =  Parameter(_rfunc, terminals=terminals)
rvec = np.array([rfunc(s) for s in states])
phi = Wrap(Bias(), terminals=terminals)

full_df = report(pmat, rvec, phi, gmfunc, lmfunc, ifunc)
df = full_df[["weights", "MSE"]]

display(full_df)

Expected Reward:
[ 1.  2.  3.  4.  5.  0.  0.]
Feature Matrix:
[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
True Values:
[ 11.66666667  21.33333333  27.          26.66666667  18.33333333   0.           0.        ]
Emphasis As Good or Better?: True


,weights,MSE,values
Least-Squares,[21.0],32.488889,"[21.0, 21.0, 21.0, 21.0, 21.0, 0.0, 0.0]"
TD,[15.0],68.488889,"[15.0, 15.0, 15.0, 15.0, 15.0, 0.0, 0.0]"
ETD,[21.0],32.488889,"[21.0, 21.0, 21.0, 21.0, 21.0, 0.0, 0.0]"


In [62]:
print(df.to_latex())

\begin{tabular}{llr}
\toprule
{} & weights &        MSE \\
\midrule
Least-Squares &  [21.0] &  32.488889 \\
TD            &  [15.0] &  68.488889 \\
ETD           &  [21.0] &  32.488889 \\
\bottomrule
\end{tabular}



## Decreasing Reward, Constant Feature

* r(s) = N-1-s for s non-terminal
* x(s) = 1 for x non-terminal

In [63]:
_rfunc = lambda x: (num_states - basis2int(x) - 1)
rfunc =  Parameter(_rfunc, terminals=terminals)
rvec = np.array([rfunc(s) for s in states])
phi = Wrap(Bias(), terminals=terminals)

full_df = report(pmat, rvec, phi, gmfunc, lmfunc, ifunc)
df = full_df[["weights", "MSE"]]

display(full_df)

Expected Reward:
[ 6.  5.  4.  3.  2.  0.  0.]
Feature Matrix:
[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]]
True Values:
[ 23.33333333  34.66666667  36.          29.33333333  16.66666667   0.           0.        ]
Emphasis As Good or Better?: True


,weights,MSE,values
Least-Squares,[28.0],52.088889,"[28.0, 28.0, 28.0, 28.0, 28.0, 0.0, 0.0]"
TD,[20.0],116.088889,"[20.0, 20.0, 20.0, 20.0, 20.0, 0.0, 0.0]"
ETD,[28.0],52.088889,"[28.0, 28.0, 28.0, 28.0, 28.0, 0.0, 0.0]"


In [64]:
print(df.to_latex())

\begin{tabular}{llr}
\toprule
{} & weights &         MSE \\
\midrule
Least-Squares &  [28.0] &   52.088889 \\
TD            &  [20.0] &  116.088889 \\
ETD           &  [28.0] &   52.088889 \\
\bottomrule
\end{tabular}



## Constant Reward, Increasing Feature

* r(s) = 1 for s non-terminal
* x(s) = s+1 for s non-terminal

In [65]:
rfunc =  Constant(1.0, terminals=terminals)
rvec = np.array([rfunc(s) for s in states])
phi = Wrap(Unary2Int(num_states), terminals=terminals)

full_df = report(pmat, rvec, phi, gmfunc, lmfunc, ifunc)
df = full_df[["weights", "MSE"]]

display(full_df)

Expected Reward:
[ 1.  1.  1.  1.  1.  0.  0.]
Feature Matrix:
[[ 1.]
 [ 2.]
 [ 3.]
 [ 4.]
 [ 5.]
 [ 0.]
 [ 0.]]
True Values:
[ 5.  8.  9.  8.  5.  0.  0.]
Emphasis As Good or Better?: True


,weights,MSE,values
Least-Squares,[1.90909090909],11.709091,"[1.90909090909, 3.81818181818, 5.72727272727, ..."
TD,[1.0],20.800000,"[1.0, 2.0, 3.0, 4.0, 5.0, 0.0, 0.0]"
ETD,[1.4],14.560000,"[1.4, 2.8, 4.2, 5.6, 7.0, 0.0, 0.0]"


## Decreasing Reward, Increasing Feature

* r(s) = N-1-s for s non-terminal
* x(s) = s+1 for s non-terminal

In [66]:
_rfunc = lambda x: (num_states - basis2int(x) - 1)
rfunc =  Parameter(_rfunc, terminals=terminals)
rvec = np.array([rfunc(s) for s in states])
phi = Wrap(Unary2Int(num_states), terminals=terminals)

full_df = report(pmat, rvec, phi, gmfunc, lmfunc, ifunc)
df = full_df[["weights", "MSE"]]

display(full_df)

Expected Reward:
[ 6.  5.  4.  3.  2.  0.  0.]
Feature Matrix:
[[ 1.]
 [ 2.]
 [ 3.]
 [ 4.]
 [ 5.]
 [ 0.]
 [ 0.]]
True Values:
[ 23.33333333  34.66666667  36.          29.33333333  16.66666667   0.           0.        ]
Emphasis As Good or Better?: True


,weights,MSE,values
Least-Squares,[7.29696969697],250.385455,"[7.29696969697, 14.5939393939, 21.8909090909, ..."
TD,[3.33333333333],423.200000,"[3.33333333333, 6.66666666667, 10.0, 13.333333..."
ETD,[4.85333333333],316.070400,"[4.85333333333, 9.70666666667, 14.56, 19.41333..."


In [67]:
print(df.to_latex())

\begin{tabular}{llr}
\toprule
{} &          weights &         MSE \\
\midrule
Least-Squares &  [7.29696969697] &  250.385455 \\
TD            &  [3.33333333333] &  423.200000 \\
ETD           &  [4.85333333333] &  316.070400 \\
\bottomrule
\end{tabular}

